In [1]:
import pandas as pd
import os
import numpy as np

____________________________

# 14 de septiembre
* Se creo la variable ‘edad_años’ para representar la edad de acuerdo a la unidad de medida y estandarizarla por años.
* Se creo la variable ‘ciclo_vital’ para categorizar los grupos etarios. 
* Se tomaron correcciones en la escritura de los nombres propios de las unidades geográficas. 
* Se mapearon las variables categóricas de número a su correspondiente significado de categoría de acuerdo a la ficha técnica SIVIGILA.
* Se toma fecha de inicio de síntomas como referencia temporal.
* Cambiar pertenencia étnica para que solo sean 3 rangos

In [2]:
files = {
    'col': r"C:\Users\Hp\DENGUE\Data\dengue_data\data_colombia.csv",
    'atl': r"C:\Users\Hp\DENGUE\Data\dengue_data\data_atlantico.csv"
}

dataframes = {}
for name, path in files.items():
    df = pd.read_csv(path, encoding='latin_1', low_memory=False)
    df.columns = df.columns.str.lower()
    dataframes[name] = df


In [3]:
# Imprimir columnas únicas por DataFrame
col_cols = set(dataframes['col'].columns)
atl_cols = set(dataframes['atl'].columns)

print("🧾 Columnas solo en Colombia:")
print(sorted(col_cols - atl_cols))

print("\n🧾 Columnas solo en Atlántico:")
print(sorted(atl_cols - col_cols))


🧾 Columnas solo en Colombia:
['cod_dpto_o', 'cod_eve', 'cod_mun_o', 'departamento']

🧾 Columnas solo en Atlántico:
['nombre_evento']


In [4]:
dataframes['col'] = dataframes['col'].drop(columns=['cod_dpto_o', 'cod_eve', 'cod_mun_o'])
dataframes['atl'] = dataframes['atl'].drop(columns=['nombre_evento'])

In [5]:
for name, df in dataframes.items():
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947810 entries, 0 to 947809
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   año                    947810 non-null  int64 
 1   area                   947810 non-null  int64 
 2   departamento           947810 non-null  object
 3   edad                   947810 non-null  int64 
 4   fecha_consulta         947717 non-null  object
 5   fecha_hospitalizacion  400814 non-null  object
 6   fecha_inicio_sintomas  947757 non-null  object
 7   fecha_nacimiento       917713 non-null  object
 8   fecha_notificacion     947810 non-null  object
 9   hospitalizado          947810 non-null  int64 
 10  muerte                 947810 non-null  int64 
 11  municipio              947810 non-null  object
 12  pertenencia_etnica     947810 non-null  int64 
 13  regimen_salud          947810 non-null  object
 14  semana                 947810 non-null  int64 
 15  

## CONVERSION DE LA EDAD

* Crear edad en años

In [6]:
for name, df in dataframes.items():
    df["edad_años"] = np.where(df["uni_med"] == 1, df["edad"],   # ya está en años
                    np.where(df["uni_med"] == 2, df["edad"] / 12,  # meses → años
                    np.where(df["uni_med"] == 3, df["edad"] / 365, # días → años
                    np.where(df["uni_med"] == 4, df["edad"] / (365*24), # horas
                    np.where(df["uni_med"] == 5, df["edad"] / (365*24*60), # minutos
                    np.where(df["uni_med"] == 0, np.nan, 
                     np.nan))))))  # por si hay otros valores
    df["edad_años"] = df["edad_años"].round().fillna(-1).astype(int)

* Crear variable ciclo vital

In [10]:
def ciclo_vital(edad):
    if edad < 0:
        return "Desconocido"
    elif edad < 1:
        return "Menor de 1 año"
    elif edad < 5:
        return "Primera infancia"
    elif edad < 10:
        return "Infancia"
    elif edad < 20:
        return "Adolescencia"
    elif edad < 60:
        return "Adulto"
    elif edad < 85:
        return "Adulto mayor"
    else:
        return "Ancianidad"
    
for name, df in dataframes.items():
    df["ciclo_vital"] = df["edad_años"].apply(ciclo_vital)

## CATEGORIZAR VARIABLES

In [11]:
mapeo = {
    'area' : {1: 'Cabecera municipal' , 2:'Centro Poblado', 3:'Rural disperso'}, 
    'hospitalizado': {1: 'Si', 2: 'No'},
    'muerte': {1: 'Vivo', 2: 'Muerto'},
    'pertenencia_etnica': {1: 'Indigena', 2: 'Rom', 3: 'Raizal', 4: 'Palenquero', 5: 'Negro', 6:'Otro'},
    'regimen_salud': {'P': 'Excepción', 'E': 'Especial', 'C': 'Contributivo', 'S': 'Subsidiado', 'N': 'No asegurado', 'I': 'Indeterminado'},
    'sexo': {'M': 'Masculino', 'F': 'Femenino'},
    'tipo_de_caso': {2: 'Probable', 3: 'Conf. por laboratorio', 5: 'Conf. por nexo'}
}

for name, df in dataframes.items():
    dataframes[name] = df.replace(mapeo)
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947810 entries, 0 to 947809
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   año                    947810 non-null  int64 
 1   area                   947810 non-null  object
 2   departamento           947810 non-null  object
 3   edad                   947810 non-null  int64 
 4   fecha_consulta         947717 non-null  object
 5   fecha_hospitalizacion  400814 non-null  object
 6   fecha_inicio_sintomas  947757 non-null  object
 7   fecha_nacimiento       917713 non-null  object
 8   fecha_notificacion     947810 non-null  object
 9   hospitalizado          947810 non-null  object
 10  muerte                 947810 non-null  object
 11  municipio              947810 non-null  object
 12  pertenencia_etnica     947810 non-null  object
 13  regimen_salud          947810 non-null  object
 14  semana                 947810 non-null  int64 
 15  

In [12]:
mapeo_etnico = {
    'Indigena': 'Indígena',
    'Negro': 'Afrocolombiano',
    'Raizal': 'Afrocolombiano',
    'Palenquero': 'Afrocolombiano',
    'Rom': 'Rom',
    'Otro': 'Otro'
}

for name, df in dataframes.items():
    df['grupo_etnico'] = df['pertenencia_etnica'].map(mapeo_etnico)

In [13]:
dataframes['atl']['grupo_etnico'].unique()

array(['Otro', 'Afrocolombiano', 'Indígena', 'Rom'], dtype=object)

In [15]:
for name, df in dataframes.items():
    dataframes[name] = df.drop(columns=['edad', 'uni_med', 'pertenencia_etnica'])

In [16]:
for name, df in dataframes.items():
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947810 entries, 0 to 947809
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   año                    947810 non-null  int64         
 1   area                   947810 non-null  object        
 2   departamento           947810 non-null  object        
 3   fecha_consulta         947717 non-null  datetime64[ns]
 4   fecha_hospitalizacion  400814 non-null  datetime64[ns]
 5   fecha_inicio_sintomas  947757 non-null  datetime64[ns]
 6   fecha_nacimiento       917713 non-null  datetime64[ns]
 7   fecha_notificacion     947810 non-null  datetime64[ns]
 8   hospitalizado          947810 non-null  object        
 9   muerte                 947810 non-null  object        
 10  municipio              947810 non-null  object        
 11  regimen_salud          947810 non-null  object        
 12  semana                 947810 non-null  int6

In [17]:
import os

# Ruta base donde quieres guardar los archivos
output_dir = r"C:\Users\Hp\DENGUE\Data\procesados"
os.makedirs(output_dir, exist_ok=True)  # crea la carpeta si no existe

# Guardar cada DataFrame
for name, df in dataframes.items():
    output_path = os.path.join(output_dir, f"{name}_pros.csv")
    df.to_csv(output_path, index=False, encoding='latin_1')
    print(f"✅ Archivo guardado: {output_path}")

✅ Archivo guardado: C:\Users\Hp\DENGUE\Data\procesados\col_pros.csv
✅ Archivo guardado: C:\Users\Hp\DENGUE\Data\procesados\atl_pros.csv
